In [1]:
import pandas as pd
import numpy as np
import geonamescache
import pycountry
import difflib
import warnings
import statistics as stat
import re

from countryinfo import CountryInfo
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
df = pd.read_csv("../data/6_shark_reorganization.csv")
df.head()

,Case Number,Year,Month,Day,Season,Time,Daytime,Country,Area,Location,Type,Activity,Injury,Deadly,Species,Size,Name,Sex,Age,Source,pdf,href,original order
0,Attack-6286,2018,6,25,0,18h00,0,United States,New Caledonia,"Oceanside, San Diego County",Boating,Padding,"No injury to occupant, outrigger canoe and pad...",N,White shark,0,julie wolfe,F,57,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0
1,Attack-6285,2018,6,18,0,14h00 -15h00,0,United States,Georgia,"St. Simon Island, Glynn County",Unprovoked,Standing,Minor injury to left thigh,N,NaN,0,adysonâ mcneely,F,11,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0
2,Attack-6284,2018,6,9,0,07h45,0,United States,Haiti,"Habush, Oahu",Invalid,Surfing,Injury to left lower leg from surfboard skeg,N,NaN,0,john denges,M,48,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0
3,Attack-6283,2018,6,8,0,NaN,0,Australia,New South Wales,Arrawarra Headland,Unprovoked,Surfing,Minor injury to lower leg,N,2 m shark,0,Unknown male,M,0,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0
4,Attack-6282,2018,6,4,0,NaN,0,Mexico,Colombia,La Ticla,Provoked,Diving,Lacerations to leg & hand shark provoked incident,N,"Tiger shark, 3m",0,gustavo ramos,M,0,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0


# Limpieza columna "Time"

In [3]:
def hour_letter (x):
    try:
        x = x.lower()
        if "nig" in x:
            return 21
        elif "aft" in x:
            return 14
        elif "eve" in x:
            return 21
        elif "mor" in x:
            return 9
        elif "mid" in x:
            return 12
        elif "dus" in x:
            return 22
        elif "dar" in x:
            return 22
        elif "noon" in x:
            return 12
        elif "sun" in x:
            return 12
        elif "dawn" in x:
            return 20
        elif "day" in x:
            return 8
        elif "a.m." in x:
            return 10
        elif "p.m." in x:
            return 18
        else:
            return x
    except:
        return x

In [4]:
def pares(x):
    try:
        matches = re.findall(r'(\d\d)', x)

        if matches and len(matches) >= 2:
            return matches[0] + "h" + matches[1]
        else:
            return x
    except:
        return x

In [5]:
def hour (time):
    try:
        if len(time) == 5:
            time = time.lower()
            time = time.split("h")

            hour = int(time[0])
            minut = int(time[1])

            if minut > 30:
                return int(hour) + 1
            elif minut <= 30:
                return int(hour)
            else:
                return "".join(time)
        else:
            return time
    except:
        return time

In [6]:
def clean(x):
    if isinstance(x, int):
        return x
    else:
        return 13

In [7]:
df.Time = df.Time.apply(pares)
df.Time = df.Time.apply(hour_letter)
df.Time = df.Time.apply(hour)
df.Time = df.Time.apply(clean)
df.Time = df.Time.apply(lambda x: 0 if x > 24 else x)

# Creación columna "Daytime"

In [8]:
def daytime (hora):
    if 6 <= hora < 12:
        return "Morning"
    elif 12 <= hora < 14:
        return "Noon"
    elif 14 <= hora < 20:
        return "Afternoon"
    else:
        return "Night"

In [9]:
df.Daytime = df.Time.apply(daytime)

# Creación columna "Season"

In [10]:
def season(month):
    if 1 <= month < 3:
        return "Winter"
    elif 3 <= month < 6:
        return "Spring"
    elif 6 <= month < 9:
        return "Summer"
    elif 9 <= month <= 12:
        return "Autumn"
        

In [11]:
df.Season = df.Month.apply(season)

In [12]:
df.Season.replace(np.NaN,"Unknown", inplace = True)

In [13]:
df.to_csv("../data/7_shark_time.csv",index=False)